# Generate release notes

This notebook automates creating a draft for the latest release notes using our custom generation script and OpenAI API. All you need to do is have a valid OpenAI API key and the URLs of latest GitHub release tags you want to include.

Release information is extracted from the release tags, and then sorted by label. Then, we put the information through OpenAI for preliminary editing. 

After running the notebook, you can locate the new generated release notes added to the `~/site/releases` folder for you to clean up further, along with a live preview of the site to get you started.

## Contents  
- [Prerequisites](#toc1_)    
- [Setting up](#toc2_)    
  - [Import release generation script](#toc2_1_)    
  - [Set up OpenAI API](#toc2_2_)    
  - [Create categories from labels](#toc2_3_)    
  - [Collect GitHub URLs](#toc2_4_)    
  - [Set the release date](#toc2_5_)    
- [Extracting PR information](#toc3_)    
  - [Create release folder and file](#toc3_1_)    
  - [Add the date to release notes ](#toc3_2_)    
  - [Set up release notes components](#toc3_3_)    
  - [Set the repository and tag name](#toc3_4_)    
  - [Extract PRs from each URL](#toc3_5_)    
  - [Load PR data](#toc3_6_)    
- [Drafting the release notes](#toc4_)    
  - [Edit the release notes body](#toc4_1_)    
  - [Fetch automated GitHub PR summary](#toc4_2_)    
  - [Edit each title](#toc4_3_)    
  - [Set labels for each PR](#toc4_4_)    
  - [Assign PR details to PR](#toc4_5_)    
  - [Assemble release notes](#toc4_6_)    
- [Adding release notes to the docs](#toc5_)    
  - [Write release notes to file](#toc5_1_)    
  - [Update sidebar](#toc5_2_)    
  - [Update index](#toc5_3_)    
- [Next steps](#toc6_)    
  - [Show files to commit](#toc6_1_)    
  - [Preview release notes](#toc6_2_)    
  - [Wrap it up](#toc6_2_)  

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1_'></a>

## Prerequisites 

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Refer our release notes guide for the full steps needed before running this notebook:</b></span>
<br></br>
<a href="https://www.notion.so/validmind/Create-Release-Notes-c4291bb92f644c5eb490cc21a8a79ca5?pvs=4" style="color: #DE257E;"><b>Create Release Notes</b></a></div>

This process assumes you have knowledge of how to work with GitHub, including managing release tags, sending commits, and creating pull requests. 

- [ ] You should be on a separate branch associated to the Story for the release notes before you begin.
- [ ] You need to have a valid `OPENAI_API_KEY` in your `.env` file. Your `.env` file should preferably live in your `documentation` repo root folder so you don't have to enter in the location of the file when the script runs.
- [ ] You need to have the GitHub release tags created and the URLs ready to be inputted.

<a id='toc2_'></a>

## Setting up 

<a id='toc2_1_'></a>

### Import release generation script 

This cell imports our custom `generate_release_objects.py` script for use with a shortened alias, including any necessary dependencies.

In [1]:
import generate_release_objects as gro

<a id='toc2_2_'></a>

### Set up OpenAI API 

First, you'll be asked to indicate the location of the `.env` file where your OpenAI API key lives so it can return the secrey key. 

**Leave blank to go with the default path:** `../.env`

In [ ]:
gro.env_location = gro.get_env_location()
gro.setup_openai_api(gro.env_location)

<a id='toc2_3_'></a>

### Create categories from labels 

This cell creates the main sections of the release notes based on the GitHub labels.

`label_hierarchy` shows the order in which updates will be shown. Leave as is to keep the existing order.

**Anything unlabeled will be shoved into `other`, which you will need to review and sort into the right categories when you edit.**

In [ ]:
label_hierarchy = ["highlight", "enhancement", "breaking-change", "deprecation", "bug", "documentation"]
gro.display_list(label_hierarchy)

<a id='toc2_4_'></a>

### Collect GitHub URLs 

Running this cell will prompt you to enter your GitHub release URLs. Keep pasting them in until you're done, then press enter again.

**Example release URL:** https://github.com/validmind/documentation/releases/tag/v2.4.4

In [ ]:
gro.github_urls = gro.collect_github_urls() 
gro.count_repos(gro.github_urls)

<a id='toc2_5_'></a>

### Set the release date 
Running this cell will prompt you to enter the desired release date. 

**The default is 3 business days from today if you leave the prompt empty.**

In [ ]:
gro.release_datetime = gro.get_release_date()
gro.formatted_release_date = gro.release_datetime.strftime("%Y-%b-%d").lower()
gro.original_release_date = gro.release_datetime.strftime("%B %-d, %Y")

### Define the unified version

Set the unified ValidMind version here associated with the release in accordance with our [customer-managed release versioning conventions](https://github.com/validmind/installation/blob/main/site/installation/Customer-managed-releases.qmd), for example `25.03.06`:

In [ ]:
unified_version = f"Unified version `{gro.input_version()}`"

<a id='toc3_'></a>

## Extracting PR information 

<a id='toc3_1_'></a>

### Create release folder and file

These lines will create a folder inside of `~/site/releases` for the new release notes to live in and sets stage for the release notes to be generated. The folder name is the release date tucked into the yearly folder, as per our convention. 

**If the directory and release file already exists, you will be prompted to confirm whether or not you want to overwrite the contents.**

In [ ]:
output_file, year = gro.create_release_folder(gro.formatted_release_date)

<a id='toc3_2_'></a>

### Add the date to release notes 
This block writes the specified date as the title (for dispaly) and date (for listing sorting) of the new release notes file.

**It will also open up the newly created `release-notes.qmd` file for you so you don't have to go looking for it.**

In [ ]:
gro.create_release_qmd(output_file, gro.original_release_date, gro.release_datetime.strftime("%Y-%m-%d"))

<a id='toc3_3_'></a>

### Set up release notes components 
`release_components` will contain all the components of the release notes in the form of a dictionary. 

Later, we will merge these components together to create the release notes. Again, anything unlabeled will be shoved into `other` at the end of the release notes. 

In [ ]:
gro.update_release_components(gro.release_components, gro.categories)

<a id='toc3_4_'></a>

### Set the repository and tag name 
This block checks every release URL and assigns its repo name, and its associated release tag name.

In [ ]:
gro.set_names(gro.github_urls)

<a id='toc3_5_'></a>

### Extract PRs from each URL 
This block gathers all the pull requests from each release URL and stores them within the URL's object data.

In [ ]:
gro.extract_urls(gro.github_urls)

<a id='toc3_6_'></a>

### Load PR data 

This block extracts information from the PR's JSON data, and stores the information in each PR's object data.

In [ ]:
gro.populate_data(gro.github_urls)

<a id='toc4_'></a>

## Drafting the release notes 

<a id='toc4_1_'></a>

### Edit the release notes body 

Using the prompt below, this block feeds the body of each PR to ChatGPT for initial editing, skipping PRs labeled as `internal`. 

**If you find that the output is not quite right, you can edit the prompt and play around with it.**

In [ ]:
editing_instructions_body = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

gro.edit_release_notes(gro.github_urls, editing_instructions_body)

<a id='toc4_2_'></a>

### Fetch automated GitHub PR summary 

Using the `github-actions` bot, we'll retrieve an auto-generated summary for comparison.

In [ ]:
summary_instructions = """ 
    Please turn this PR Summary into a summary for release notes, according to the following guidelines:
    - Use simple and neutral language in the active voice.
    - Change from numbered list format to paragraph-style text.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    """

gro.auto_summary(gro.github_urls, summary_instructions)

<a id='toc4_3_'></a>

### Edit each title 
This block does the same as `Edit the release notes body` for the titles of each PR. The output below will show:
- The original PR title
- The title after some algorithmic changes
- The title after ChatGPT edits it

**If you find that it's not good after editing with ChatGPT, feel free to edit the prompt below.**

In [ ]:
editing_instructions_title = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

gro.edit_titles(gro.github_urls, editing_instructions_title)

<a id='toc4_4_'></a>

### Set labels for each PR 
This block takes the label data from each PR and assigns it to the PR.

In [ ]:
gro.set_labels(gro.github_urls)

<a id='toc4_5_'></a>

### Assign PR details to PR 
This block compiles all the data we found earlier for each PR into one place. 

In [ ]:
gro.assign_details(gro.github_urls)

<a id='toc4_6_'></a>

### Assemble release notes 
Now, we can take all the details we compiled above and append them to our final release notes components. 

Since we want to show features in order of importance, we sort by the priority of the label.

In [ ]:
release_components = gro.assemble_release(gro.github_urls, label_hierarchy)

<a id='toc5_'></a>

## Adding release notes to the docs 

<a id='toc5_1_'></a>

### Write release notes to file 
Now that `release_components` contains everything we need for the release notes, we can write it to our release notes file.

This step also appends our single-source includes file `/releases/_how-to-upgrade.qmd` automatically to the bottom of the page. 

In [ ]:
gro.release_output(output_file, release_components, gro.label_to_category)
gro.upgrade_info(output_file)

<a id='toc5_2_'></a>

### Update sidebar 
This block will go into our releases `_sidebar.yaml` file and add the new release notes so it shows up on the sidenav of the docsite under the "Releases" section. 

In [ ]:
gro.update_release_sidebar(gro.release_datetime, year)

<a id='toc5_3_'></a>

### Update index 
This block will go into our `index.qmd` file and add the new release notes so it shows up in the "Latest Releases" section, as well as remove the oldest release so that the tiles are uniform. 

In [ ]:
gro.update_index_qmd(gro.release_datetime, year)

<a id='toc6_'></a>

## Next steps 

<a id='toc6_1_'></a>

### Show files to commit 

This block shows you the files changed or added by the notebook you need to commit — it should omit any files in `release-scripts/`. 

In [ ]:
%%bash
cd ../
git status | grep -v 'release-scripts/'

<a id='toc6_2_'></a>

### Preview release notes 
Run this cell to preview the site, including the new release notes you just generated. 

In [ ]:
%%bash
cd ../site
quarto preview

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>When you're done with the preview, please restart the kernel.</b></span>
<br></br>
Make sure that when you commit your changes to remote, do NOT include the changes in the <code>~/release-scripts/</code> folder as that would commit changed notebook with the output cells.</div>

<a id='toc6_3_'></a>

### Wrap it up

**You may want to send a commit up to remote before you begin editing so you have a backup.**

- [ ] At the end of the release notes file, make sure that the notebook appended the `_how-to_upgrade.qmd` include to the very bottom.
- [ ] Make sure that the link to the new release notes displays as expected in the sidebar under `About > Releases`. 
- [ ] Double-check that the `Latest Releases` section on our main documentation landing page includes the new release and looks uniform.
- [ ] Edit the new release notes in accordance with our [internal guide](https://www.notion.so/validmind/Create-Release-Notes-c4291bb92f644c5eb490cc21a8a79ca5?pvs=4) to make sure you've completed all the requirements.
- [ ] Make sure any relevant files are committed to remote in preparation for your PR!
